In [ ]:
!pip install pulp

In [ ]:
import numpy as np
import pandas as pd
import math
import random
import csv

import pulp as pp

from params import *
from functions import *

import os
os.makedirs('results/', exist_ok = True) 

In [ ]:
results_SEP = pd.read_csv('results/results_online_SEP.txt', header = None)
results_SHR = pd.read_csv('results/results_online_SHR.txt', header = None)
results_SHR_CLUSTER = pd.read_csv('results/results_online_SHR_CLUSTER.txt', header = None)
results_SHR_ROBUST = pd.read_csv('results/results_online_SHR_ROBUST.txt', header = None)
results_SHR_FULL = pd.read_csv('results/results_online_SHR_FULL.txt', header = None)

In [ ]:
f = open('results/results_online_OPTIMAL.txt', 'w+')
f_null = open('results/results_online_NULL.txt', 'w+')

for SEED in range(0,10):

    set_seed(SEED)
    data_incoming = pd.read_pickle(f'data/data_incoming_{SEED}.pickle')
    
    N_incoming = max(max(results_SEP[results_SEP[0] == SEED][4]), 
                     results_SHR[3][SEED], results_SHR_CLUSTER[3][SEED], 
                     results_SHR_ROBUST[3][SEED], 
                     results_SHR_FULL[3][SEED])
    
    objective_vector = np.zeros(N_incoming*len(passenger_incentive_list)*len(driver_incentive_list))
    budget_vector = np.zeros(N_incoming*len(passenger_incentive_list)*len(driver_incentive_list))
    
    # decision variable
    x = [pp.LpVariable(f'x{i}',cat='Binary') for i in range(0, N_incoming*len(passenger_incentive_list)*len(driver_incentive_list))]
    x_null = np.zeros(N_incoming*len(passenger_incentive_list)*len(driver_incentive_list))
    m = pp.LpProblem(sense=pp.LpMaximize)
    
    k = 0
    
    for n in range(0,N_incoming):

        fare = data_incoming.loc[n, 'fare']
        compensation = data_incoming.loc[n, 'compensation']
        
        m += (pp.lpSum(x[k:k+len(passenger_incentive_list)*len(driver_incentive_list)]) == 1)
        
        for i in range(0, len(passenger_incentive_list)):
            for j in range(0, len(driver_incentive_list)):
                
                if (i == 0 and j == 0):
                    x_null[k] = 1
                
                # treat the underlying true probability as input
                true_prob = data_incoming.loc[n, f'true_prob_{i}_{j}']
                objective_vector[k] = true_prob
                budget_vector[k] = true_prob * (fare*passenger_incentive_list[i] + compensation*driver_incentive_list[j])
                
                if (fare*(1-passenger_incentive_list[i]) < compensation*(1+driver_incentive_list[j])):
                    m += (x[k] == 0)
                    
                k += 1

    m += pp.lpDot(objective_vector,x)
    m += (pp.lpDot(budget_vector,x) <= B)
    
    # obtain the optimal solution
    m.solve()
    
    f.write(f'{SEED}, OPTIMAL, {pp.value(pp.lpDot(objective_vector,x))}\n')
    f_null.write(f'{SEED}, NULL, {np.dot(objective_vector,x_null)}\n')
    
f.close()
f_null.close()